In [54]:
import Pkg
pkgs = [
    "Eisenia",
    "Random",
    "Revise",
    "BioSequences",
    "Test",
    "LightGraphs",
    "Primes",
    "Statistics",
    "Plots",
    "uCSV",
    "DataFrames",
    "HTTP",
    "CodecZlib",
    "FASTX",
    "ProgressMeter"
]

for pkg in pkgs
    try
        Pkg.add(pkg)
    catch
#         # tried to install an unregistered local package
    end
    eval(Meta.parse("import $pkg"))
end

  Resolving package versions...
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes

In [2]:
TASK = "making-neo4j-graphs"
DATE = "2021-03-18"
DIR = "$(homedir())/$(DATE)-$(TASK)"
if !isdir(DIR)
    mkdir(DIR)
end
cd(DIR)

In [3]:
function get_genome_from_ftp_directory(ftp_path)
    fasta_url = "$(ftp_path)/$(basename(ftp_path))_genomic.fna.gz"
    fasta_buffer = CodecZlib.GzipDecompressorStream(IOBuffer(HTTP.get(fasta_url).body))
    return collect(FASTX.FASTA.Reader(fasta_buffer))
end

get_genome_from_ftp_directory (generic function with 1 method)

In [4]:
db = "genbank"
assembly_summary_url = "https://ftp.ncbi.nlm.nih.gov/genomes/$(db)/assembly_summary_$(db).txt"
assembly_summary_file = "$(DIR)/$(basename(assembly_summary_url))"
if !isfile(assembly_summary_file)
    open(assembly_summary_file, "w") do io
        assembly_summary_response = HTTP.get(assembly_summary_url)
        write(io, assembly_summary_response.body)
    end
end
assembly_summary_table = DataFrames.DataFrame(uCSV.read(assembly_summary_file, delim='\t', header=2, skipmalformed=true)...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCA_000001215.4,PRJNA13812,SAMN02803731,,reference genome
2,GCA_000001405.28,PRJNA31257,,,reference genome
3,GCA_000001515.5,PRJNA13184,SAMN02981217,AACZ00000000.4,na
4,GCA_000001545.3,PRJNA20869,SAMN02981238,ABGA00000000.1,na
5,GCA_000001635.9,PRJNA20689,,,reference genome
6,GCA_000001735.2,PRJNA10719,SAMN03081427,,reference genome
7,GCA_000001765.3,PRJNA10626,SAMN00779672,AADE00000000.2,na
8,GCA_000001895.4,PRJNA10629,SAMN02808228,AABR00000000.7,na
9,GCA_000001905.1,PRJNA12569,SAMN02953622,AAGU00000000.3,representative genome


In [5]:
genome_size_table_url = "ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/species_genome_size.txt.gz"
genome_size_table_file = "$(DIR)/$(basename(genome_size_table_url))"
if !isfile(genome_size_table_file)
    open(genome_size_table_file, "w") do io
        genome_size_table_response = HTTP.get(genome_size_table_url)
        write(io, genome_size_table_response.body)
    end
end
genome_size_table_buffer = CodecZlib.GzipDecompressorStream(open(genome_size_table_file))
genome_size_table = DataFrames.DataFrame(uCSV.read(genome_size_table_buffer, header=1, delim='\t')...)

,#species_taxid,min_ungapped_length,max_ungapped_length,expected_ungapped_length
,Int64,Int64,Int64,Int64
1,9,318000,956000,636876
2,24,2950000,6050000,4500062
3,34,7405000,11109000,9257167
4,56,5994000,15000000,11989871
5,69,4891000,7337000,6113769
6,114,7212000,10820000,9015893
7,122,6222000,9334000,7777997
8,124,5461000,9315000,7387897
9,139,813000,1786000,1299537


In [6]:
joint_table = DataFrames.innerjoin(assembly_summary_table, genome_size_table, on=["species_taxid" => "#species_taxid"])
sort!(joint_table, "expected_ungapped_length")

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604
9,GCA_000851485.1,,,,reference genome,154834


In [8]:
# 112903 on refseq
# 795 on genbank!

In [9]:
group = first(DataFrames.groupby(joint_table, "taxid"))

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCA_002829805.1,,,,na,230604
2,GCA_003972145.1,,,,na,230604
3,GCA_003972165.1,,,,na,230604
4,GCA_003972185.1,,,,na,230604
5,GCA_003972205.1,,,,na,230604
6,GCA_003972225.1,,,,na,230604
7,GCA_003972245.1,,,,na,230604
8,GCA_003972265.1,,,,na,230604


In [10]:
# how many groups are there?

In [11]:
show(group, allcols=true)

8×26 SubDataFrame
│ Row │ # assembly_accession │ bioproject │ biosample │ wgs_master │
│     │ String               │ String     │ String    │ String     │
├─────┼──────────────────────┼────────────┼───────────┼────────────┤
│ 1   │ GCA_002829805.1      │            │           │            │
│ 2   │ GCA_003972145.1      │            │           │            │
│ 3   │ GCA_003972165.1      │            │           │            │
│ 4   │ GCA_003972185.1      │            │           │            │
│ 5   │ GCA_003972205.1      │            │           │            │
│ 6   │ GCA_003972225.1      │            │           │            │
│ 7   │ GCA_003972245.1      │            │           │            │
│ 8   │ GCA_003972265.1      │            │           │            │

│ Row │ refseq_category │ taxid  │ species_taxid │ organism_name     │
│     │ String          │ Int64  │ Int64         │ String            │
├─────┼─────────────────┼────────┼───────────────┼───────────────────┤
│ 1   │ n

In [12]:
taxid = group[1, "taxid"]

230604

In [15]:
if !occursin("cypher-shell", ENV["PATH"])
    ENV["PATH"] = "/Users/cameronprybol/Software/cypher-shell:$(ENV["PATH"])"
end

"/Users/cameronprybol/Software/cypher-shell:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Users/cameronprybol/miniconda/bin:/Users/cameronprybol/.gem/ruby/2.6.0/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"

In [13]:
function list_databases()
    io = open(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto "show databases"`)
    return DataFrames.DataFrame(uCSV.read(io, header=1, quotes='"', encodings=Dict("FALSE" => false, "TRUE" => true))...)
end

list_databases (generic function with 1 method)

In [14]:
function create_database(database_id)
    current_databases = list_databases()
    if database_id in current_databases[!, "name"]
        return
    else
        run(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto "create database $(database_id)"`)
    end
end

create_database (generic function with 1 method)

In [20]:
function cypher(database_id, cmd)
    run(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database $(database_id) --format auto $(cmd)`)
end

cypher (generic function with 1 method)

In [16]:
database_id = "taxid$(taxid)"

"taxid230604"

In [17]:
create_database(database_id)

In [18]:
ftp_path = group[1, "ftp_path"]

"ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/002/829/805/GCA_002829805.1_ASM282980v1"

In [19]:
accession = group[1, "# assembly_accession"]

"GCA_002829805.1"

In [64]:
for (accession, ftp_path) in DataFrames.eachrow(group[!, ["# assembly_accession", "ftp_path"]])
    for fasta in get_genome_from_ftp_directory(ftp_path)
        identifier = FASTX.identifier(fasta)
        description = FASTX.description(fasta)
        sequence = FASTX.sequence(fasta)
        cypher_call = "MERGE (g:GENOME {accession: '$(accession)'})<-[s:SOURCE]-(f:FASTA {identifier: '$(identifier)', description: '$(description)', sequence: '$(sequence)'}) RETURN *"
        cypher(database_id, cypher_call)
    end
end

f, g, s
(:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete cds"}), (:GENOME {accession: "GCA_002829805.1"}), [:SOURCE]
f, g, s

In [21]:
graphs = []
for record in get_genome_from_ftp_directory(group[1, "ftp_path"])
    graph = Eisenia.KmerGraph(BioSequences.DNAMer{7}, record)
    push!(graphs, graph)
end
graphs

1-element Array{Any,1}:
 Eisenia.KmerGraph{BioSequences.Mer{BioSequences.DNAAlphabet{2},7}}({743, 774} undirected simple Int64 graph, Dict{LightGraphs.SimpleGraphs.SimpleEdge{Int64},Array{Eisenia.EdgeEvidence,1}}(Edge 344 => 426 => [Eisenia.EdgeEvidence("AY247793.1", 271)],Edge 467 => 521 => [Eisenia.EdgeEvidence("AY247793.1", 165)],Edge 594 => 635 => [Eisenia.EdgeEvidence("AY247793.1", 229)],Edge 21 => 77 => [Eisenia.EdgeEvidence("AY247793.1", 784)],Edge 209 => 547 => [Eisenia.EdgeEvidence("AY247793.1", 494)],Edge 318 => 479 => [Eisenia.EdgeEvidence("AY247793.1", 600)],Edge 28 => 565 => [Eisenia.EdgeEvidence("AY247793.1", 377)],Edge 221 => 412 => [Eisenia.EdgeEvidence("AY247793.1", 352)],Edge 241 => 405 => [Eisenia.EdgeEvidence("AY247793.1", 193)],Edge 707 => 738 => [Eisenia.EdgeEvidence("AY247793.1", 54)]…), BioSequences.Mer{BioSequences.DNAAlphabet{2},7}[AAAACAA, AAAACCC, AAAACGA, AAAACTA, AAAACTG, AAAATAT, AAACAAG, AAACCCT, AAACCTA, AAACGAG  …  TGGATAA, TGGCCAA, TGGTGAA, TGTGTAA, T

In [22]:
graph = first(graphs)

Eisenia.KmerGraph{BioSequences.Mer{BioSequences.DNAAlphabet{2},7}}({743, 774} undirected simple Int64 graph, Dict{LightGraphs.SimpleGraphs.SimpleEdge{Int64},Array{Eisenia.EdgeEvidence,1}}(Edge 344 => 426 => [Eisenia.EdgeEvidence("AY247793.1", 271)],Edge 467 => 521 => [Eisenia.EdgeEvidence("AY247793.1", 165)],Edge 594 => 635 => [Eisenia.EdgeEvidence("AY247793.1", 229)],Edge 21 => 77 => [Eisenia.EdgeEvidence("AY247793.1", 784)],Edge 209 => 547 => [Eisenia.EdgeEvidence("AY247793.1", 494)],Edge 318 => 479 => [Eisenia.EdgeEvidence("AY247793.1", 600)],Edge 28 => 565 => [Eisenia.EdgeEvidence("AY247793.1", 377)],Edge 221 => 412 => [Eisenia.EdgeEvidence("AY247793.1", 352)],Edge 241 => 405 => [Eisenia.EdgeEvidence("AY247793.1", 193)],Edge 707 => 738 => [Eisenia.EdgeEvidence("AY247793.1", 54)]…), BioSequences.Mer{BioSequences.DNAAlphabet{2},7}[AAAACAA, AAAACCC, AAAACGA, AAAACTA, AAAACTG, AAAATAT, AAACAAG, AAACCCT, AAACCTA, AAACGAG  …  TGGATAA, TGGCCAA, TGGTGAA, TGTGTAA, TGTTGAA, TTAATAA, TTACCAA,

In [58]:
cypher_command = join(["MERGE (:DNAMer {sequence: \"$(kmer)\"})" for kmer in graph.kmers], '\n')
cypher(database_id, cypher_command)

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database taxid230604 --format auto 'MERGE (:DNAMer {sequence: "AAAACAA"})
MERGE (:DNAMer {sequence: "AAAACCC"})
MERGE (:DNAMer {sequence: "AAAACGA"})
MERGE (:DNAMer {sequence: "AAAACTA"})
MERGE (:DNAMer {sequence: "AAAACTG"})
MERGE (:DNAMer {sequence: "AAAATAT"})
MERGE (:DNAMer {sequence: "AAACAAG"})
MERGE (:DNAMer {sequence: "AAACCCT"})
MERGE (:DNAMer {sequence: "AAACCTA"})
MERGE (:DNAMer {sequence: "AAACGAG"})
MERGE (:DNAMer {sequence: "AAACTAC"})
MERGE (:DNAMer {sequence: "AAACTCT"})
MERGE (:DNAMer {sequence: "AAACTGG"})
MERGE (:DNAMer {sequence: "AAACTTG"})
MERGE (:DNAMer {sequence: "AAAGAAT"})
MERGE (:DNAMer {sequence: "AAAGACC"})
MERGE (:DNAMer {sequence: "AAAGATC"})
MERGE (:DNAMer {sequence: "AAAGTCA"})
MERGE (:DNAMer {sequence: "AAAGTGC"})
MERGE (:DNAMer {sequence: "AAAGTTA"})
MERGE (:DNAMer {sequence: "AAATAAA"})
MERGE (:DNAMer {sequence: "AAATACC"})
MERGE (:DNAMer {sequence: "AAATATT

In [63]:
# cyper_commands = []
ProgressMeter.@showprogress for edge_evidence in collect(graph.edge_evidence)
    edge = first(edge_evidence)
    evidence = last(edge_evidence)
    src_kmer = graph.kmers[edge.src]
    dst_kmer = graph.kmers[edge.dst]
    possible_orientations = map(bool -> Eisenia.assess_path_orientations([edge.src, edge.dst], graph.kmers, bool), [true, false])
    possible_orientations = filter!(x -> x != nothing, possible_orientations)
    orientations = first(possible_orientations)
    string_orienations = string(Int.(orientations))
    cypher_command = 
    """
    MATCH (src:DNAMer {sequence: "$(src_kmer)"})
    MATCH (dst:DNAMer {sequence: "$(dst_kmer)"})
    MERGE (src)-[edge:EDGE {orientations: $(string_orienations)}]->(dst)
    RETURN *
    """
#     push!(cyper_commands, cypher_command)
    cypher(database_id, cypher_command)
end

Progress:  12%|████▉                                    |  ETA: 0:12:49

dst, edge, src
(:DNAMer {sequence: "CATTGGC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "ATTGGCG"})
dst, edge, src
(:DNAMer {sequence: "CTATACC"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "CCTATAC"})
dst, edge, src
(:DNAMer {sequence: "GGATTGA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "GATTGAC"})
dst, edge, src
(:DNAMer {sequence: "AATAAAG"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AAATAAA"})
dst, edge, src
(:DNAMer {sequence: "CTGTTGC"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "AGCAACA"})
dst, edge, src
(:DNAMer {sequence: "CGAAGCA"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "ATGCTTC"})
dst, edge, src
(:DNAMer {sequence: "GAACAAC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AACAACC"})
dst, edge, src
(:DNAMer {sequence: "CATACGC"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "AGCGTAT"})
dst, edge, src
(:DNAMer {sequence: "CAGTCAC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AGT

Progress:  24%|█████████▊                               |  ETA: 0:10:53

dst, edge, src
(:DNAMer {sequence: "TGCCTAA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "GCCTAAC"})
dst, edge, src
(:DNAMer {sequence: "TAGGGGA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AGGGGAC"})
dst, edge, src
(:DNAMer {sequence: "TGGACCA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "ATGGACC"})
dst, edge, src
(:DNAMer {sequence: "CTGCTCA"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "ATGAGCA"})
dst, edge, src
(:DNAMer {sequence: "CCCTTGC"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "CAAGGGG"})
dst, edge, src
(:DNAMer {sequence: "TGCAGCA"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "GCTGCAA"})
dst, edge, src
(:DNAMer {sequence: "GAAAACC"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AGAAAAC"})
dst, edge, src
(:DNAMer {sequence: "CAGCATA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AGCATAC"})
dst, edge, src
(:DNAMer {sequence: "AATGACT"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "AAG

Progress:  36%|██████████████▋                          |  ETA: 0:09:14

...

dst, edge, src
(:DNAMer {sequence: "TGCAGCA"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "GCTGCAC"})
dst, edge, src
(:DNAMer {sequence: "CAATACG"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AATACGT"})
dst, edge, src
(:DNAMer {sequence: "TGTTGAA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "GTTGAAA"})
dst, edge, src
(:DNAMer {sequence: "GGGGAGA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AGGGGAG"})
dst, edge, src
(:DNAMer {sequence: "CTAACCA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "CCTAACC"})
dst, edge, src
(:DNAMer {sequence: "CTAAAAC"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "CGTTTTA"})
dst, edge, src
(:DNAMer {sequence: "GGACTCC"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "GAGTCCC"})
dst, edge, src
(:DNAMer {sequence: "AGCAAAG"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "ACTTTGC"})
dst, edge, src
(:DNAMer {sequence: "GAACCAC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AAC

Progress:  95%|██████████████████████████████████████▉  |  ETA: 0:00:51

dst, edge, src
(:DNAMer {sequence: "CTTATGC"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "AGCATAA"})
dst, edge, src
(:DNAMer {sequence: "CTCCGAA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "ACTCCGA"})
dst, edge, src
(:DNAMer {sequence: "TAAGGAA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "GTAAGGA"})
dst, edge, src
(:DNAMer {sequence: "CAGGCAC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AGGCACA"})
dst, edge, src
(:DNAMer {sequence: "GGGTAAC"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "GGGGTAA"})
dst, edge, src
(:DNAMer {sequence: "GGCATAC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "GCATACA"})
dst, edge, src
(:DNAMer {sequence: "ATTCACC"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AATTCAC"})
dst, edge, src
(:DNAMer {sequence: "ACGGTAT"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "AATACCG"})
dst, edge, src
(:DNAMer {sequence: "TTTAAAA"}), [:EDGE {orientations: [1, 0]}], (:DNAMer {sequence: "GTT

Progress: 100%|█████████████████████████████████████████| Time: 0:16:47


dst, edge, src
(:DNAMer {sequence: "CGTTTTA"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "AAAACGA"})
dst, edge, src
(:DNAMer {sequence: "ATCCTCG"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AATCCTC"})
dst, edge, src
(:DNAMer {sequence: "CAGGAGA"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "AGGAGAT"})
dst, edge, src
(:DNAMer {sequence: "TGGAGAA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "CTGGAGA"})
dst, edge, src
(:DNAMer {sequence: "CATGCCC"}), [:EDGE {orientations: [0, 0]}], (:DNAMer {sequence: "ATGCCCC"})
dst, edge, src
(:DNAMer {sequence: "AAGGTTG"}), [:EDGE {orientations: [0, 1]}], (:DNAMer {sequence: "AACCTTC"})
dst, edge, src
(:DNAMer {sequence: "ACCCTTA"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AACCCTT"})
dst, edge, src
(:DNAMer {sequence: "AAACTAC"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AAAACTA"})
dst, edge, src
(:DNAMer {sequence: "AGTGAAG"}), [:EDGE {orientations: [1, 1]}], (:DNAMer {sequence: "AAG

In [75]:
# cyper_commands = []
ProgressMeter.@showprogress for edge_evidence in collect(graph.edge_evidence)
    edge = first(edge_evidence)
    evidence = last(edge_evidence)
    src_kmer = graph.kmers[edge.src]
    dst_kmer = graph.kmers[edge.dst]
    unique_sources = unique(map(e -> e.record_identifier, evidence))
    for unique_source in unique_sources
        cypher_command = 
        """
        MATCH (src:DNAMer {sequence: "$(src_kmer)"})
        MATCH (dst:DNAMer {sequence: "$(dst_kmer)"})
        MATCH (f:FASTA) WHERE f.identifier = "$(unique_source)"
        MERGE (src)-[src_f:CONTAINED_IN]->(f)
        MERGE (dst)-[dst_f:CONTAINED_IN]->(f)
        RETURN *
        """
        cypher(database_id, cypher_command)
    end
end

Progress:   1%|▌                                        |  ETA: 0:15:04

dst, dst_f, f, src, src_f
(:DNAMer {sequence: "CATTGGC"}), [:CONTAINED_IN], (:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete

Progress:   2%|█                                        |  ETA: 0:14:49

dst, dst_f, f, src, src_f
(:DNAMer {sequence: "CATGGGG"}), [:CONTAINED_IN], (:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete

Progress:   4%|█▌                                       |  ETA: 0:14:38

...

dst, dst_f, f, src, src_f
(:DNAMer {sequence: "ACGTCAG"}), [:CONTAINED_IN], (:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete

Progress:  99%|████████████████████████████████████████▊|  ETA: 0:00:06

dst, dst_f, f, src, src_f
(:DNAMer {sequence: "ATACGTC"}), [:CONTAINED_IN], (:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete

Progress: 100%|█████████████████████████████████████████| Time: 0:14:35


dst, dst_f, f, src, src_f
(:DNAMer {sequence: "AGACGTA"}), [:CONTAINED_IN], (:FASTA {identifier: "AY247793.1", sequence: "CCACGTCTAGCTGCTGACGTTAAATGCAGCCGGGTGGTAATGCGTATTAATATTTCAACAACATGAATAAGCGCATTAATAATAATCGGAGAACCATGAGATCACGTAGGGGACGCGGTAGGACAATGGGATCTAATCTCATTCCTTACGCCAATTCACCAGTCCCTATACCATATACACCACCCGTTACCCCAGTCACCGTCATTAGTAATCCTCGGAAAACTACTTGGATTGACATTGATCTTTCAAGTGAAGAGTCCGGGATTTACACATTGGCGGTTGGGTCATACCGTAATAGGATTACTAAACTTGGTCCATCTAAACCTAACTTTATTATTGAGAAGGTTGCAGCATACGCTGCACCAGGAGATTATAAGGTTGTTCTCAATGACTTTAAAACTGGCATACAAGTCGTTGATGAAGGCTCTTATGCTCATAGAGCCGCAGTAGGTATTCTTTATCCACCTGCTGCACAAATATTTTACGGTATTTCAGCAACAGGCACACTCAATACTATCACTACCACTGCTAAAGATCCAGTTCCAGTGGTTCGTGCCTTGGTAACATATTGGGACTCCGAACAGTAATGAGCAGGTTCTATGCTTCGAACTAAAACAAGCAATGAGTTCTAGCTCCGAACTAAAACGAGCCAAACTCTCGCATCTGAGTATAAATCATGCCCCATGATCCTCGCATCGTATCGCACTTTGCTGGAGAAAACCCTTATGGCCAATTGTGGCAAGGGGAGACGTATTGGTTAGGCATTATTCCGGGTGCGTCGATAAATAAAGACCTT", description: "Macrobrachium rosenbergii XSV virus capsid protein gene, complete